In [ ]:
from typing import List, Tuple
import os
import cv2
import json
from collections import defaultdict

folder = "kfilter"
output_folder = "output_videos"
results_json = "conteo_resultados.json"

# Crear carpeta de salida si no existe
os.makedirs(output_folder, exist_ok=True)

rest_of_videos: List[str] = [os.path.join(folder, video) for video in [
    "01-1.mp4", "02-1.mp4", "03-1.mp4", "04-1.mp4", "05-1.mp4",
    "06-1.mp4", "07-1.mp4", "08-1.mp4", "09-1.mp4", "10-1.mp4",
    "11-1.mp4", "12-1.mp4", "13-1.mp4", "14-1.mp4", "15-1.mp4",
    "16-1.mp4", "17-1.mp4", "18-1.mp4", "19-1.mp4", "20-1.mp4",
    "21-1.mp4", "22-1.mp4", "23-1.mp4", "24-1.mp4", "25-1.mp4",
    "28-1.mp4", "29-1.mp4", "30-1.mp4", "31-1.mp4", "32-1.mp4",
    "33-1.mp4", "34-1.mp4", "35-1.mp4", "36-1.mp4", "37-1.mp4",
    "38-1.mp4", "39-1.mp4", "41-1.mp4", "42-1.mp4", "43-1.mp4"
]]

# Variables globales para la configuración de línea
line_points = []
current_video_path = ""

def mouse_callback(event, x, y, flags, param):
    """Callback para capturar los puntos de la línea de conteo"""
    global line_points
    
    if event == cv2.EVENT_LBUTTONDOWN:
        if len(line_points) < 2:
            line_points.append((x, y))
            print(f"Punto {len(line_points)} seleccionado: ({x}, {y})")
            if len(line_points) == 2:
                print("Línea configurada. Presiona 'ENTER' para continuar o 'r' para reiniciar")

def configure_line(video_path: str) -> Tuple[Tuple[int, int], Tuple[int, int]]:
    """Permite al usuario configurar la línea de conteo para un video"""
    global line_points
    line_points = []
    
    cap = cv2.VideoCapture(video_path)
    ret, frame = cap.read()
    cap.release()
    
    if not ret:
        print(f"Error al leer el video: {video_path}")
        return None
    
    # Reducir tamaño si es muy grande
    height, width = frame.shape[:2]
    if width > 1280:
        scale = 1280 / width
        frame = cv2.resize(frame, (int(width * scale), int(height * scale)))
    
    window_name = f"Configura la línea - {os.path.basename(video_path)}"
    cv2.namedWindow(window_name)
    cv2.setMouseCallback(window_name, mouse_callback)
    
    print(f"\n--- Configurando: {os.path.basename(video_path)} ---")
    print("Haz clic en 2 puntos para definir la línea de conteo")
    print("Presiona 'r' para reiniciar, 'ENTER' para confirmar, 'q' para saltar video")
    
    while True:
        display_frame = frame.copy()
        
        # Dibujar puntos y línea
        for i, point in enumerate(line_points):
            cv2.circle(display_frame, point, 5, (0, 255, 0), -1)
            cv2.putText(display_frame, str(i+1), (point[0]+10, point[1]), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        
        if len(line_points) == 2:
            cv2.line(display_frame, line_points[0], line_points[1], (0, 255, 0), 2)
        
        cv2.imshow(window_name, display_frame)
        
        key = cv2.waitKey(1) & 0xFF
        if key == 13 and len(line_points) == 2:  # ENTER
            break
        elif key == ord('r'):  # Reiniciar
            line_points = []
            print("Línea reiniciada. Selecciona 2 puntos nuevamente")
        elif key == ord('q'):  # Saltar
            cv2.destroyAllWindows()
            return None
    
    cv2.destroyAllWindows()
    return tuple(line_points)

def point_above_line(point, line_start, line_end):
    """Determina si un punto está por encima de la línea (dirección del cruce)"""
    x, y = point
    x1, y1 = line_start
    x2, y2 = line_end
    
    # Producto cruzado para determinar el lado
    return (x2 - x1) * (y - y1) - (y2 - y1) * (x - x1) > 0

def process_video(video_path: str, line: Tuple[Tuple[int, int], Tuple[int, int]]) -> int:
    """Procesa el video y cuenta los cruces de la línea"""
    cap = cv2.VideoCapture(video_path)
    
    # Propiedades del video
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    # Configurar video de salida
    output_path = os.path.join(output_folder, os.path.basename(video_path))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    
    # Sustracción de fondo
    bg_subtractor = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=50, detectShadows=False)
    
    # Variables de conteo
    count = 0
    tracked_objects = {}
    next_object_id = 0
    
    line_start, line_end = line
    
    print(f"Procesando: {os.path.basename(video_path)}")
    frame_count = 0
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        frame_count += 1
        
        # Detección de movimiento
        fg_mask = bg_subtractor.apply(frame)
        
        # Limpieza de ruido
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
        fg_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_OPEN, kernel)
        fg_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_CLOSE, kernel)
        
        # Encontrar contornos
        contours, _ = cv2.findContours(fg_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        current_centroids = []
        
        for contour in contours:
            area = cv2.contourArea(contour)
            if area > 500:  # Filtrar objetos pequeños
                M = cv2.moments(contour)
                if M["m00"] != 0:
                    cx = int(M["m10"] / M["m00"])
                    cy = int(M["m01"] / M["m00"])
                    current_centroids.append((cx, cy))
        
        # Tracking simple y detección de cruces
        new_tracked = {}
        for centroid in current_centroids:
            matched = False
            for obj_id, obj_data in tracked_objects.items():
                prev_centroid = obj_data['centroid']
                distance = ((centroid[0] - prev_centroid[0])**2 + (centroid[1] - prev_centroid[1])**2)**0.5
                
                if distance < 50:  # Umbral de matching
                    # Verificar cruce de línea
                    prev_side = point_above_line(prev_centroid, line_start, line_end)
                    curr_side = point_above_line(centroid, line_start, line_end)
                    
                    if prev_side != curr_side and not obj_data['counted']:
                        count += 1
                        obj_data['counted'] = True
                        print(f"  Frame {frame_count}: ¡Cruce detectado! Total: {count}")
                    
                    new_tracked[obj_id] = {'centroid': centroid, 'counted': obj_data['counted']}
                    matched = True
                    break
            
            if not matched:
                new_tracked[next_object_id] = {'centroid': centroid, 'counted': False}
                next_object_id += 1
        
        tracked_objects = new_tracked
        
        # Dibujar en el frame
        cv2.line(frame, line_start, line_end, (0, 255, 0), 3)
        cv2.putText(frame, f"Conteo: {count}", (10, 50), 
                   cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 3)
        
        # Dibujar centroides
        for obj_data in tracked_objects.values():
            cv2.circle(frame, obj_data['centroid'], 5, (0, 0, 255), -1)
        
        out.write(frame)
    
    cap.release()
    out.release()
    
    print(f"Conteo final: {count}")
    print(f"Video guardado en: {output_path}\n")
    
    return count

# Procesar todos los videos
results = {}

for video_path in rest_of_videos:
    if not os.path.exists(video_path):
        print(f"Video no encontrado: {video_path}")
        continue
    
    # Configurar línea para este video
    line = configure_line(video_path)
    
    if line is None:
        print(f"Video saltado: {os.path.basename(video_path)}\n")
        continue
    
    # Procesar video
    count = process_video(video_path, line)
    
    # Guardar resultado
    video_name = os.path.basename(video_path)
    results[video_name] = {
        "conteo": count,
        "linea": {"inicio": line[0], "fin": line[1]}
    }

# Guardar JSON con resultados
with open(results_json, 'w', encoding='utf-8') as f:
    json.dump(results, f, indent=2, ensure_ascii=False)

print(f"\n{'='*50}")
print(f"Procesamiento completado!")
print(f"Resultados guardados en: {results_json}")
print(f"Videos procesados en: {output_folder}")
print(f"{'='*50}")